# Kaggle- Multilayered Perceptron (MLP) implemention on MNIST dataset
Training the model to achieve best accuracy with more number of parameters.

In [26]:
import os
from pathlib import Path
import torch
from torch.utils.data import TensorDataset ,DataLoader
from torch import nn,optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
PATH=Path("../input/digit-recognizer")
print(os.listdir("../input/digit-recognizer"))

['test.csv', 'train.csv', 'sample_submission.csv']


## Loading Data 

In [27]:
train=pd.read_csv(PATH/'train.csv')
test=pd.read_csv(PATH/'test.csv')
train.shape,test.shape

((42000, 785), (28000, 784))

## Extracting Input and Target Variable

In [28]:
x=train.drop("label",axis=1)
y=np.array(train['label'])
x.shape,y.shape

((42000, 784), (42000,))

## Train -Test Split -Pytorch

In [29]:
torch_X_train = torch.from_numpy(x.values).type(torch.FloatTensor)/255   # Normalizing the X values
torch_y_train = torch.from_numpy(y).type(torch.LongTensor)
myDataset = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
valid_no  = int(0.2 * len(myDataset))
# so divide the data into trainset and testset
trainSet,testSet = torch.utils.data.random_split(myDataset,(len(myDataset)-valid_no,valid_no))
print(f"len of trainSet {len(trainSet)} , len of testSet {len(testSet)}")
batch_size=64
train_loader  = DataLoader(trainSet , batch_size=batch_size ,shuffle=True) 
test_loader  = DataLoader(testSet , batch_size=batch_size ,shuffle=True)

len of trainSet 33600 , len of testSet 8400


## Building Network

In [30]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 392)
        self.fc2 = nn.Linear(392, 196)
        self.fc3 = nn.Linear(196, 98)
        self.fc4 = nn.Linear(98, 10)

        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        # Now with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        # output so no dropout here
        x = F.log_softmax(self.fc4(x), dim=1)

        return x
        
model=Network()
optimizer=optim.SGD(model.parameters(),momentum = 0.9, lr = 0.01)
criterion=nn.NLLLoss()

## Accuracy 

In [35]:
epochs=9
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in train_loader:
        optimizer.zero_grad()
        log_ps=model(images)
        loss=criterion(log_ps,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            model.eval()
            for images,labels in test_loader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels)
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.mean(equals.type(torch.FloatTensor))
        model.train()
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))    

Epoch: 1/8..  Training Loss: 0.022..  Test Loss: 0.081..  Test Accuracy: 0.978
Epoch: 2/8..  Training Loss: 0.022..  Test Loss: 0.074..  Test Accuracy: 0.978
Epoch: 3/8..  Training Loss: 0.020..  Test Loss: 0.083..  Test Accuracy: 0.978
Epoch: 4/8..  Training Loss: 0.021..  Test Loss: 0.082..  Test Accuracy: 0.978
Epoch: 5/8..  Training Loss: 0.018..  Test Loss: 0.083..  Test Accuracy: 0.979
Epoch: 6/8..  Training Loss: 0.017..  Test Loss: 0.083..  Test Accuracy: 0.978
Epoch: 7/8..  Training Loss: 0.016..  Test Loss: 0.084..  Test Accuracy: 0.979
Epoch: 8/8..  Training Loss: 0.016..  Test Loss: 0.088..  Test Accuracy: 0.978


## Save our model

In [ ]:
print("Best model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')

# Reference 

[Introduction to Pytorch-Udacity](https://github.com/udacity/deep-learning-v2-pytorch/tree/master/intro-to-pytorch)